In [ ]:
import pandas as pd
import numpy as np
import os
import time
import glob
from random import sample

from scipy.io import wavfile, savemat
from scipy.fft import fft, fftfreq
from scipy import signal
from scipy.fft import fftshift
from scipy.io import loadmat
from sklearn.model_selection import train_test_split

from IPython.display import Audio, display
from IPython.display import clear_output

import librosa
import mne
import pymatreader
import yasa
import seaborn as sns
import pyxdf

from mne.time_frequency import psd_array_multitaper
from keras.callbacks import ModelCheckpoint
from mne.decoding import CSP
from scipy.integrate import simps
from yasa import sliding_window
from lazypredict.Supervised import LazyClassifier

import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
# %matplotlib qt

%matplotlib inline
import plotly.io as pio
pio.renderers.default='notebook'

# Steps in this notebook
1. Filter the data [Done in EEGLAB]
2. Single-subject ERP
3. Grand average ERP


In [ ]:
data_path = "/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/PreprocessedData/Preprocess HAPPE"
exclusion = ["FP01", "FP10"]
os.chdir(data_path)

In [ ]:
sf = 1000
ch_type = "eeg"
nchan = 64
channels = [f'E{n:1}' for n in range(1, nchan+1)]

In [ ]:
mne.channels.get_builtin_montages()
chans = mne.channels.make_standard_montage('GSN-HydroCel-64_1.0')
info = mne.create_info(channels, sfreq=sf, ch_types=ch_type)
info.set_montage(chans)

In [ ]:
def get_action_events(events):
    new_keys_dict = {}
    for key in events[1].keys():
        if "ActionBeg" in key:
            new_keys_dict[key] = events[1][key]
    new_event_times = [row for row in events[0] if row[2] in new_keys_dict.values()] 
    return (new_event_times, new_keys_dict)

In [ ]:
eeg_df_main = pd.DataFrame()

for files in glob.glob("*.set"):
    if files[:-4] not in exclusion:
        arr = mne.io.read_raw_eeglab(files, verbose=0)
        eeg_data = arr.to_data_frame()
        events = mne.events_from_annotations(arr, verbose=0)
        
        # Extracting events of interest
        new_events = get_action_events(events)
        
        epochs = mne.Epochs(arr, new_events[0], event_id=new_events[1], tmin=-0.1, tmax=0.999,  baseline=None, verbose=0)
        epoch_df = epochs.to_data_frame()
        
        n_epochs = len(np.unique(epoch_df['epoch']))
        
        # Transforming epoch dataframe to dataframe of interets
        all_epochs_df = pd.DataFrame()
        ctr = 1
        for n_epoch in range(n_epochs):
            eeg_only = epoch_df[epoch_df["epoch"] == n_epoch].reset_index().iloc[:, 4:].T
            label = epoch_df[epoch_df["epoch"] == n_epoch]["condition"].iloc[0]
            eeg_only["HandPos"] = label.split("-")[1]
            eeg_only["Hand"] = label.split("-")[2]
            eeg_only["Action"] = label.split("-")[3]
            eeg_only["Subject"] = files.split(".")[0]
            eeg_only["Channel"] = [x[1:] for x in eeg_only.index]
            eeg_only["Trial"] = ctr
            eeg_only.reset_index()
            ctr += 1

            all_epochs_df = pd.concat([all_epochs_df, eeg_only])

        eeg_df_main = pd.concat([eeg_df_main, all_epochs_df])
        print(eeg_df_main.shape)

In [ ]:
eeg_df_main = eeg_df_main.rename(columns={"Component": "Channel"})
eeg_df_main

In [ ]:
eeg_df_main

In [ ]:
df = pd.DataFrame(eeg_df_main, columns=list(range(1100))+["HandPos", "Hand", "Action", "Subject", "Channel", "Trial"])
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/CleanSep/")
df.to_csv("cleaned_data_asr.csv", index=False, header=list(range(1100))+["handPos", "Hand", "Action", "Subject", "Channel", "Trial"])
df

## Save trials as MAT file as well.

In [ ]:
dataset.head()

In [ ]:
subjets = np.unique(dataset.Subject)
subjects

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/IntData/ASR/matv1")
for sub in subjects:
    for palm in ["palmDown", "palmIn", "palmUp"]:
        for oc in ["Open", "Close"]:
            for hand in ["Left", "Right"]:
                cond_eeg = dataset[
                        (dataset["Subject"] == sub) &
                        (dataset["handPos"] == palm) &
                        (dataset["Action"] == oc) &
                        (dataset["Hand"] == hand)
                    ]
                trials = np.unique(cond_eeg.Trial)
                for trial in trials:
                    p_eeg = cond_eeg[cond_eeg["Trial"] == trial]
                    save_name = sub+"-"+palm+"-"+hand+"-"+oc+"-"+str(trial)+".mat"
                    savemat(save_name, mdict={"EEG": p_eeg.to_numpy() })

# ERP Analysis
Subject-wise average and ERP analysis
1. Electrode-wise
2. Average of electrodes in the ROI: 

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/CleanSep/")
dataset = pd.read_csv("cleaned_data_asr.csv")
dataset

In [ ]:
np.unique(dataset.iloc[:, -2], return_counts=True)

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/PreprocessedData/Preprocessedv2/")
arr = mne.io.read_raw_eeglab("P01.set", verbose=0)
channels = arr.ch_names#np.array([[x] for x in ])
events = mne.events_from_annotations(arr, verbose=0)
print(channels)

In [ ]:
chans.plot()
plt.show()

In [ ]:
# All subjects
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Plots/ASR/Subject-wise plots")
for sub in subjects:
    conditions_dict = {}
    conds_trial_ctr = {}
    for x in events[1].keys():
        if "ActionBeg" in x:
            conditions_dict[x[10:]] = []
            conds_trial_ctr[x[10:]] = 0

    # Single subject ERP
    conds = conditions_dict
    cond_ctr = conds_trial_ctr

    for palm in ["palmDown", "palmIn", "palmUp"]:
        for oc in ["Open", "Close"]:
            for hand in ["Left", "Right"]:
                cond_data = dataset[(dataset["handPos"] == palm) & 
                        (dataset["Action"] == oc) & 
                        (dataset["Hand"] == hand) & 
                        (dataset["Subject"] == sub)]
                epochs = np.unique(cond_data.iloc[:, -1])

                for i in epochs:
                    test_sub_trial = cond_data[cond_data["Trial"] == i]
                    test_sub_trial.index = test_sub_trial.iloc[:, -2]
                    name = palm + "-" + hand + "-" + oc 
                    
                    if test_sub_trial.shape[1] == 0:
                        continue

                    if len(conds[name]) > 0:
                        conds[name] += test_sub_trial.iloc[:, :-6].to_numpy()
                    else:
                        conds[name] = test_sub_trial.iloc[:, :-6].to_numpy()

                    cond_ctr[name] += 1
    cond_avg = {}
    for key in conds.keys():
        cond_avg[key] = conds[key] / cond_ctr[key]

    channels = np.unique(dataset[dataset["Subject"] == sub].iloc[:, -2])

    for key in cond_avg.keys():
        fig = px.line(pd.DataFrame(cond_avg[key].T, columns=channels), title=sub + "-" + key)
        fig.write_image(sub + "-" + key + ".png") 
#         fig.show()

In [ ]:
# Average of ROI channels for subject-wise ERP
# All subjects
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Plots/ASR/Subject-wise ROI")
for sub in subjects:
    conditions_dict = {}
    conds_trial_ctr = {}
    for x in events[1].keys():
        if "ActionBeg" in x:
            conditions_dict[x[10:]] = []
            conds_trial_ctr[x[10:]] = 0
            
    sub_channels = np.unique(dataset[dataset["Subject"] == sub].iloc[:, -2])
    roi_chan = list(pd.read_csv("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/ChannelLocations/roi-grasp-small.csv", header=None))
    current_roi = [x for x in sub_channels if x in roi_chan]
    
    # Single subject ERP
    conds = conditions_dict
    cond_ctr = conds_trial_ctr

    for palm in ["palmDown", "palmIn", "palmUp"]:
        for oc in ["Open", "Close"]:
            for hand in ["Left", "Right"]:
                cond_data = dataset[(dataset["handPos"] == palm) & 
                        (dataset["Action"] == oc) & 
                        (dataset["Hand"] == hand) & 
                        (dataset["Subject"] == sub)]
                epochs = np.unique(cond_data.iloc[:, -1])

                for i in epochs:
                    test_sub_trial = cond_data[cond_data["Trial"] == i]
                    test_sub_trial.index = test_sub_trial.iloc[:, -2]
                    name = palm + "-" + hand + "-" + oc 
                    
                    if test_sub_trial.shape[1] == 0:
                        continue

                    if len(conds[name]) > 0:
                        conds[name] += test_sub_trial.iloc[:, :-6].to_numpy()
                    else:
                        conds[name] = test_sub_trial.iloc[:, :-6].to_numpy()

                    cond_ctr[name] += 1
    cond_avg = {}
    for key in conds.keys():
        cond_avg[key] = conds[key] / cond_ctr[key]

    cond_roi = {}
    for key in cond_avg.keys():
        current_roi = [x for x in sub_channels if x in roi_chan]
        cond_roi[key] = np.mean(cond_avg[key][tuple(current_roi), :], axis=0)
    
    for palm in ["palmDown", "palmIn", "palmUp"]:
        for hand in ["Left", "Right"]:
            cond = palm+"-"+hand 
            fig = px.line(
                pd.DataFrame(data=[cond_roi[cond+"-Open"], cond_roi[cond+"-Close"]], index=["Open", "Close"]).T,
                title=cond+"-"+sub
            )
            fig.write_image(cond+"-"+sub + ".png") 
#             fig.show()

In [ ]:
# Grand-average ERPs with ROI channels' average
all_subj_cond = {}
subjects = np.unique(dataset.Subject)
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/Plots/ASR/Grand Average")

for sub in subjects:
    conditions_dict = {}
    conds_trial_ctr = {}
    for x in events[1].keys():
        if "ActionBeg" in x:
            conditions_dict[x[10:]] = []
            conds_trial_ctr[x[10:]] = 0
    
    sub_channels = np.unique(dataset[dataset["Subject"] == sub].iloc[:, -2])
    roi_chan = list(pd.read_csv("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/ChannelLocations/roi-grasp-small.csv", header=None))
    current_roi = [x for x in sub_channels if x in roi_chan]
    
    # Single subject ERP
    conds = conditions_dict
    cond_ctr = conds_trial_ctr
    
    for palm in ["palmDown", "palmIn", "palmUp"]:
        for oc in ["Open", "Close"]:
            for hand in ["Left", "Right"]:
                cond_data = dataset[(dataset["handPos"] == palm) & 
                        (dataset["Action"] == oc) & 
                        (dataset["Hand"] == hand) & 
                        (dataset["Subject"] == sub)]
                epochs = np.unique(cond_data.iloc[:, -1])

                for i in epochs:
                    test_sub_trial = cond_data[cond_data["Trial"] == i]
                    test_sub_trial.index = test_sub_trial.iloc[:, -2]
                    name = palm + "-" + hand + "-" + oc 
                    
                    if test_sub_trial.shape[1] == 0:
                        continue

                    if len(conds[name]) > 0:
                        conds[name] += test_sub_trial.iloc[:, :-6].to_numpy()
                    else:
                        conds[name] = test_sub_trial.iloc[:, :-6].to_numpy()

                    cond_ctr[name] += 1

    cond_avg = {}
    for key in conds.keys():
        cond_avg[key] = conds[key] / cond_ctr[key]

    cond_roi = {}
    for key in cond_avg.keys():
        current_roi = [x for x in sub_channels if x in roi_chan]
        cond_roi[key] = np.mean(cond_avg[key][tuple(current_roi), :], axis=0)
    
    for key in cond_roi.keys():
        if key in all_subj_cond:
            all_subj_cond[key] += cond_roi[key] 
        else:
            all_subj_cond[key] = cond_roi[key] 
#         print(all_subj_cond[key])
    
for palm in ["palmDown", "palmIn", "palmUp"]:
    for hand in ["Left", "Right"]:
        cond = palm+"-"+hand 
        fig = px.line(
            pd.DataFrame(data=[cond_roi[cond+"-Open"], cond_roi[cond+"-Close"]], index=["Open", "Close"]).T,
            title=cond
        )
        fig.write_image(cond+".png") 
    
        #fig.show()


# Feature Extraction Methods
1. Power spectral density [band-wise and entire band] Running...
2. Raw data
3. ~ICA Activation~
4. Spectopo graphs

# Experiments
1. Left vs Right
2. In vs Up vs Down
3. Open vs Close

    3.1 Left hand and right hand
    
    3.2 Inwards, upwards, and downwards directions

# Validation Techniques
1. K-fold cross validation
2. Predict right hand from left and right hemishpere and vice-versa
3. Channel-wise prediction
4. Compare classification performance on ROI vs non-ROI electrodes.

# Prediction Methods
1. Single-subject

    1.1 EEGNET
    
    1.2 Machine Learning
    
    1.3 Meta Learning
    
    1.4 CNN (spectopo)
    
    1.5 [Graph Convolutional Neural Network](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9976236)

In [ ]:
os.chdir("/mnt/sda1/shivam/Thesis/Grasp Experiment/Data/CleanSep/")
dataset = pd.read_csv("cleaned_data_asr.csv")
dataset.head()

# Power Spectral Density
1. Delta
2. Theta
3. Alpha
4. Beta
5. Gamma
6. Entire Band

In [ ]:
subjects = np.unique(dataset.Subject)
X, y_oc, y_lr, y_pos, subjs = [], [], [], [], []
grasp_labels = {"Open": 0, "Close": 1}
hand_labels = {"Left": 0, "Right": 1}
pos_dict = {"palmDown": 0, "palmUp": 1, "palmIn": 2}

for sub in subjects:
    sub_channels = np.unique(dataset[dataset["Subject"] == sub].iloc[:, -2])
    for palm in ["palmDown", "palmIn", "palmUp"]:
        for oc in ["Open", "Close"]:
            for hand in ["Left", "Right"]:
                cond_data = dataset[(dataset["handPos"] == palm) & 
                        (dataset["Action"] == oc) & 
                        (dataset["Hand"] == hand) & 
                        (dataset["Subject"] == sub)]
                epochs = np.unique(cond_data.iloc[:, -1])

                for i in epochs:
                    test_sub_trial = cond_data[cond_data["Trial"] == i]
                    test_sub_trial.index = test_sub_trial.iloc[:, -2]
                     
                    X.append(test_sub_trial.iloc[:, :-6].to_numpy())
                    subjs.append(test_sub_trial.iloc[0, -3])
                    y_oc.append(grasp_labels[test_sub_trial.iloc[0, -4]])
                    y_lr.append(hand_labels[test_sub_trial.iloc[0, -5]])
                    y_pos.append(pos_dict[test_sub_trial.iloc[0, -6]])

X = np.array(X)
y_oc = np.array(y_oc) 
y_lr = np.array(y_lr)
y_pos = np.array(y_pos) 
subjs = np.array(subjs)
X.shape, y_oc.shape, y_lr.shape, y_pos.shape, subjs.shape

In [ ]:
X[0].shape

In [ ]:
# Multitaper method
def bandpower_multitaper(data, sf, method, band, relative=False):
    band = np.asarray(band)
    low, high = band

    if method == 'multitaper':
        psd_trial, freqs = psd_array_multitaper(data, sf, adaptive=True,
                                                normalization='full', verbose=0)
    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find index of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using parabola (Simpson's rule)
    bp = simps(psd_trial[idx_band], dx=freq_res)

    return bp

In [ ]:
def create_bandpower_features(epochs, frequency):
    bandpower_multitaper_EEG = []
    # Iterating over each subject [20]
    for epoch in epochs:
        # Iterating over each song per subject [30]
        bands_video = []
        no_channels = epoch.shape[0]
        input_brainwaves = epoch
        # Iterating over each channel [14]
        for k in range(no_channels):
            bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
                                                    band=[0.5, 4], relative=False))
            bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
                                                    band=[4, 7], relative=False))
            bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
                                                    band=[8, 13], relative=False))
            bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
                                                    band=[14, 30], relative=False))
            bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
                                                    band=[31, 50], relative=False))
            bands_video.append(bandpower_multitaper(input_brainwaves[k,:], sf=frequency, method='multitaper',
                                                    band=[8, 40], relative=False))
      
        bandpower_multitaper_EEG.append(bands_video)

    bandpower_multitaper_EEG = np.array(bandpower_multitaper_EEG)

    return bandpower_multitaper_EEG

In [ ]:
sf = 1000
window = 1
psd_features = []
for x in X:
    epoch = sliding_window(x, sf=sf, window=window, step=window)[1]
    psd = create_bandpower_features(epoch, sf)
    psd_features.append(psd)
    print(len(psd_features))
psd_features = np.array(psd_features)
psd_features.shape

In [ ]:
psd_features.shape